# Tiny model -- classify northern or southern california

In [3]:
import torchvision.models as models
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
import numpy as np
import matplotlib.pyplot as plt

torch.cuda.is_available()

True

In [6]:
import torch
import webdataset as wds

BATCH_SIZE = 16

dataset = wds.WebDataset("/home/fyhuang/LocalProjects/img2loc_dataset/outputs/california_tiny.tar")\
    .decode("torchrgb").to_tuple("jpg", "json")
    #.shuffle(1000)\

# Transform the labels to either ncal or scal
def to_nslabel(sample):
    inputs, target = sample
    target = dict(target)
    #target["label"] = "ncal" if target["district"].lower() in ["alameda", "contra costa", "marin", "napa", "san francisco", "san mateo", "santa clara", "solano", "sonoma", "sacramento"] else "scal"
    target["label"] = 0 if target["district"].lower() in ["alameda", "contra costa", "marin", "napa", "san francisco", "san mateo", "santa clara", "solano", "sonoma", "sacramento"] else 1
    target["label"] = torch.tensor([target["label"]])
    return inputs, target

def to_img_label(sample):
    img, target = sample
    label = 0 if target["district"].lower() in ["alameda", "contra costa", "marin", "napa", "san francisco", "san mateo", "santa clara", "solano", "sonoma", "sacramento"] else 1
    # TODO: this is where we transform the image
    return img, label

dataset = dataset.map(to_img_label)
dataset = dataset.batched(BATCH_SIZE)

# Visualize a few loaded samples
#dataloader = torch.utils.data.DataLoader(dataset, num_workers=0, batch_size=None)
dataloader = wds.WebLoader(dataset, batch_size=None, num_workers=0)
#dataloader = dataloader.unbatched().shuffle(1000).batched(BATCH_SIZE)

for inputs, targets in dataloader:
    print(inputs.shape, targets.shape)
    break

torch.Size([16, 3, 640, 640]) torch.Size([16])


In [7]:
device = torch.device("cuda:0")

def train_model(model, criterion, optimizer, scheduler, dataloader, num_epochs=25):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        model.train()
        running_batches = 0
        running_loss = 0.0
        running_corrects = 0

        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, targets)

            loss.backward()
            optimizer.step()

            running_batches += 1
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == targets.data)

            if running_batches % (1000//BATCH_SIZE) == 0:
                print(f"Trained on {running_batches*BATCH_SIZE} examples")

        scheduler.step()

        total_batches = running_batches
        epoch_loss = running_loss / (total_batches*BATCH_SIZE)
        epoch_acc = running_corrects.double() / (total_batches*BATCH_SIZE)

        print('Loss: {:.4f} Acc: {:.4f}'.format(
            epoch_loss, epoch_acc))

    return model


# Transfer learning on mobilenetv3
model = models.mobilenet_v3_large(weights="IMAGENET1K_V2")
# Freeze the model
for param in model.parameters():
    param.requires_grad = False

model.classifier = nn.Linear(model.classifier[0].in_features, 2)
torch.nn.init.xavier_uniform_(model.classifier.weight)

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
train_model(model, criterion, optimizer, scheduler, dataloader, num_epochs=4)

print("Done")

Epoch 0/3
----------
Trained on 992 examples
Trained on 1984 examples
Trained on 2976 examples
Trained on 3968 examples
Trained on 4960 examples
Trained on 5952 examples
Trained on 6944 examples
Loss: 0.6749 Acc: 0.6052
Epoch 1/3
----------
Trained on 992 examples
Trained on 1984 examples
Trained on 2976 examples
Trained on 3968 examples
Trained on 4960 examples
Trained on 5952 examples
Trained on 6944 examples
Loss: 0.6478 Acc: 0.6296
Epoch 2/3
----------
Trained on 992 examples
Trained on 1984 examples
Trained on 2976 examples
Trained on 3968 examples
Trained on 4960 examples
Trained on 5952 examples
Trained on 6944 examples
Loss: 0.6320 Acc: 0.6451
Epoch 3/3
----------
Trained on 992 examples
Trained on 1984 examples
Trained on 2976 examples
Trained on 3968 examples
Trained on 4960 examples
Trained on 5952 examples
Trained on 6944 examples
Loss: 0.6209 Acc: 0.6558
Done
